This is sourced from the first session of ESRI's **Spatial Data Science** Massive Open Online Course (MOOC).

# Data engineering

This notebook describes the process to download and prepare United States presidential election data. You will address missing values, reformat data types, and restructure the format of a table.

***

## Load and prepare data

To download and prepare the election data, you will use ArcPy, the ArcGIS API for Python, and a Pandas dataframe. First, you will import these modules to use them. Then, you will create a variable for the United States county election data and use this variable to read the data into a Pandas dataframe.

In [1]:
import arcgis
import pandas

In [6]:
data_df = pandas.read_csv('./countypres2016.csv')
print(data_df.shape)
data_df.head()

(9474, 11)


,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2016,Alabama,AL,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973,20190722
1,2016,Alabama,AL,Autauga,1001.0,President,Donald Trump,republican,18172.0,24973,20190722
2,2016,Alabama,AL,Autauga,1001.0,President,Other,NaN,865.0,24973,20190722
3,2016,Alabama,AL,Baldwin,1003.0,President,Hillary Clinton,democrat,18458.0,95215,20190722
4,2016,Alabama,AL,Baldwin,1003.0,President,Donald Trump,republican,72883.0,95215,20190722


***

## Handle missing data 

The election data includes a records that are missing data in the FIPS field. This missing data is referred to as null values. You will identify how many rows have null values and create a new dataframe that does not include them.
![Null Values](img/null_values.gif "Null Values")

In [7]:
# Perform a query on the dataframe using the loc function and the necessary field name.
data_df.loc[data_df['FIPS'].isnull()]  # We can use the isnull function built in to Pandas to find the records with null FIPS.

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
9462,2016,Connecticut,NaN,Statewide writein,NaN,President,Hillary Clinton,democrat,NaN,5056,20190722
9463,2016,Maine,NaN,Maine UOCAVA,NaN,President,Hillary Clinton,democrat,3017.0,5056,20190722
9464,2016,Alaska,NaN,District 99,NaN,President,Hillary Clinton,democrat,274.0,5056,20190722
9465,2016,Rhode Island,NaN,Federal Precinct,NaN,President,Hillary Clinton,democrat,637.0,5056,20190722
9466,2016,Connecticut,NaN,Statewide writein,NaN,President,Donald Trump,republican,NaN,5056,20190722
9467,2016,Maine,NaN,Maine UOCAVA,NaN,President,Donald Trump,republican,648.0,5056,20190722
9468,2016,Alaska,NaN,District 99,NaN,President,Donald Trump,republican,40.0,5056,20190722
9469,2016,Rhode Island,NaN,Federal Precinct,NaN,President,Donald Trump,republican,53.0,5056,20190722
9470,2016,Connecticut,NaN,Statewide writein,NaN,President,Other,NaN,NaN,5056,20190722
9471,2016,Maine,NaN,Maine UOCAVA,NaN,President,Other,NaN,321.0,5056,20190722


In [8]:
# Determine how many rows are in the table
rowcount = data_df.shape[0]

# Determine how many rows have null FIPS 
null_fips_rowcount = data_df.loc[data_df['FIPS'].isnull()].shape[0]

# Calculate how much of the data this represents as a percentage
percentage_null_fips = round((null_fips_rowcount / rowcount) * 100, 2)

# Use a print statement to report this information
print("There were "+str(null_fips_rowcount)+" records with null FIPS values in the data.\nThis amounts to " +str(percentage_null_fips)+"% of the available data.")

There were 12 records with null FIPS values in the data.
This amounts to 0.13% of the available data.


In [9]:
# Use the notnull function and the loc function to create a new dataframe without null FIPS records
data_df = data_df.loc[data_df['FIPS'].notnull()]

***

## Explore and handle data types

In reviewing your data, you notice that the FIPS field is considered a numeric field instead of a string. As a result, leading zeroes in the FIPS values have been removed. The resulting FIPS values only have four characters instead of five. You will determine how many records are missing leading zeroes and add, or append, the missing zero.
![fix_truncated_zeroes](img/trunc_zeroes.gif "Fix Truncated Zeroes")

In [10]:
# Get the first five records of the table
data_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2016,Alabama,AL,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973,20190722
1,2016,Alabama,AL,Autauga,1001.0,President,Donald Trump,republican,18172.0,24973,20190722
2,2016,Alabama,AL,Autauga,1001.0,President,Other,NaN,865.0,24973,20190722
3,2016,Alabama,AL,Baldwin,1003.0,President,Hillary Clinton,democrat,18458.0,95215,20190722
4,2016,Alabama,AL,Baldwin,1003.0,President,Donald Trump,republican,72883.0,95215,20190722


In [11]:
data_df.dtypes

year                int64
state              object
state_po           object
county             object
FIPS              float64
office             object
candidate          object
party              object
candidatevotes    float64
totalvotes          int64
version             int64
dtype: object

### Note
My FIPS column was read as <code>float64</code> so I had to modify some code here.  If yours was read as<code>int</code> or <code>object</code>, you may have to undo my changes.

In [13]:
# Check how many records have a FIPS value with four characters

# commented out code for folks who don't have floats
# trunc_df = data_df.loc[data_df['FIPS'].astype('str').str.len() == 4]
trunc_df = data_df.loc[data_df['FIPS'].astype('int').astype('str').str.len() == 4]
trunc_data_per = (trunc_df.shape[0] / data_df.shape[0])*100

# Use another print statement (using the f format key) to report this information
print(f"{round(trunc_data_per, 2)}% of data ({trunc_df.shape[0]} rows) has truncated FIPS values.")

10.37% of data (981 rows) has truncated FIPS values.


The following cell creates a function in python that adds a leading zero to the FIPS value if it only has four characters.  

In [14]:
# Define a helper function to fix truncated zeros, with one parameter: the value to be processed
def fix_trunc_zeros(val):
    # ===== I added this to fix float issue ====
    if type(val)==float:
        val = int(val)
        
    # Use an if statement to check if there are four characters in the string representation of the value
    if len(str(val)) == 4:
        # If this is the case, return the value with an appended "0" in the front
        return "0"+str(int(val))
    # Otherwise...
    else:
        # Return the value itself
        return str(int(val))

In [15]:
# Test helper function with truncated value
fix_trunc_zeros(7042.0)  # You should see an appended zero: "07042"


'07042'

In [16]:
# Run helper function on the FIPS field using the apply and lambda method 
data_df['FIPS'] = data_df['FIPS'].apply(lambda x: fix_trunc_zeros(x))

# Print information on the operation performed, and show the first few records to confirm it worked
print(f"{round(trunc_data_per, 2)}% of data ({trunc_df.shape[0]} rows) had truncated FIPS IDs corrected.")
data_df.head()

10.37% of data (981 rows) had truncated FIPS IDs corrected.


,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2016,Alabama,AL,Autauga,01001,President,Hillary Clinton,democrat,5936.0,24973,20190722
1,2016,Alabama,AL,Autauga,01001,President,Donald Trump,republican,18172.0,24973,20190722
2,2016,Alabama,AL,Autauga,01001,President,Other,NaN,865.0,24973,20190722
3,2016,Alabama,AL,Baldwin,01003,President,Hillary Clinton,democrat,18458.0,95215,20190722
4,2016,Alabama,AL,Baldwin,01003,President,Donald Trump,republican,72883.0,95215,20190722


***

## Reformat the table structure

Currently, each record in the table corresponds to a candidate and their votes in a county. You need to reformat the table so that each record corresponds to each county, with fields showing the votes for different candidates in that election year. 
It is possible to do this using the [Pivot Table geoprocessing tool](https://pro.arcgis.com/en/pro-app/tool-reference/data-management/pivot-table.htm) or Excel pivot tables, but Python may make it easier to automate and share.
The animation below illustrates the steps in restructuring the table:
1. Set a few fields aside, "locking" them from the table pivot. 
2. Pivot the table using the remaining fields.
3. Rename the pivoted fields to designate each party. 
4. Bring the locked fields back to the table. 
The following code cell performs these steps.
![reformat_table](img/reformat_table.gif "Reformat Table")


In [21]:
# Set an index using mulitple fields, which "locks" these fields before the table pivots
# Use the built-in groupby function for the FIPS and year fields, which you use to group the data by candidate
# Use unstack to perform the table pivot, which will rotate the table and turn rows into columns
df_out = data_df.set_index(['FIPS', 
                            'year', 
                            'county', 
                            'state', 
                            'state_po', 
                            'office', 
                            data_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

# Use the indexes for the columns to set column names (Ex: candidate_1, candidate_2, votes_1, votes_2, etc.)
df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)

# Rename columns 
df_out = df_out.rename(columns={"candidate_1": "candidate_dem",
                                "candidatevotes_1": "votes_dem",
                                "candidate_2": "candidate_gop",
                                "candidatevotes_2": "votes_gop",
                                "totalvotes_1": "votes_total",
                                "state_po": "state_abbrev"
                                })

# Keep only the necessary columns
df_out = df_out[["candidate_dem", "votes_dem",
                 "candidate_gop", "votes_gop",
                 "votes_total"]]

# Remove the multiindex since we no longer need these fields to be "locked" for the pivot
df_out.reset_index(inplace=True)

# Print out the first few records to confirm everything worked
df_out.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588


Pandas has three powerful capabilities that helped you perform this operation: 
- The ability to set an index using multiple fields, which acts as our "locking" mechanism. 
- The ability to unstack (or pivot) a table.
- The ability to perform an operation using a "groupby" function.

## Calculate additional columns

You will use the values from the updated table to add additional columns of information, such as the number of votes for a non major party, the percentage of voters for each party, and so on. Each column is referred to as an attribute of the dataset.

##### Calculate an attribute for the total votes for non major party

In [22]:
# Calculate votes that did not choose the Democrat or Republican party
df_out['votes_other'] = df_out['votes_total'] - (df_out['votes_dem'] + df_out['votes_gop'])
df_out.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973,865.0
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215,3874.0
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469,144.0
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819,207.0
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588,573.0


##### Calculate additional attributes

In [23]:
# Calculate voter share attributes
df_out['voter_share_major_party'] = (df_out['votes_dem'] + df_out['votes_gop']) / df_out['votes_total']
df_out['voter_share_dem'] = df_out['votes_dem'] / df_out['votes_total']
df_out['voter_share_gop'] = df_out['votes_gop'] / df_out['votes_total']
df_out['voter_share_other'] = df_out['votes_other'] / df_out['votes_total']

# Calculate raw difference attributes
df_out['rawdiff_dem_vs_gop'] = df_out['votes_dem'] - df_out['votes_gop']
df_out['rawdiff_gop_vs_dem'] = df_out['votes_gop'] - df_out['votes_dem']
df_out['rawdiff_dem_vs_other'] = df_out['votes_dem'] - df_out['votes_other']
df_out['rawdiff_gop_vs_other'] = df_out['votes_gop'] - df_out['votes_other']
df_out['rawdiff_other_vs_dem'] = df_out['votes_other'] - df_out['votes_dem']
df_out['rawdiff_other_vs_gop'] = df_out['votes_other'] - df_out['votes_gop']

# Calculate percent difference attributes
df_out['pctdiff_dem_vs_gop'] = (df_out['votes_dem'] - df_out['votes_gop']) / df_out['votes_total']
df_out['pctdiff_gop_vs_dem'] = (df_out['votes_gop'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_dem_vs_other'] = (df_out['votes_dem'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_gop_vs_other'] = (df_out['votes_gop'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_other_vs_dem'] = (df_out['votes_other'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_other_vs_gop'] = (df_out['votes_other'] - df_out['votes_gop']) / df_out['votes_total']

df_out.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,...,5071.0,17307.0,-5071.0,-17307.0,-0.489969,0.489969,0.203059,0.693028,-0.203059,-0.693028
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,...,14584.0,69009.0,-14584.0,-69009.0,-0.571601,0.571601,0.153169,0.724770,-0.153169,-0.724770
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,...,4727.0,5310.0,-4727.0,-5310.0,-0.055688,0.055688,0.451524,0.507212,-0.451524,-0.507212
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,...,1667.0,6531.0,-1667.0,-6531.0,-0.551536,0.551536,0.189024,0.740560,-0.189024,-0.740560
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,...,1583.0,22286.0,-1583.0,-22286.0,-0.809090,0.809090,0.061865,0.870955,-0.061865,-0.870955


***

## Geoenable the data

You will eventually use this data in a spatial analysis. This means that the data needs to include location information to determine where the data is located on a map. You will geoenable the data, or add location to the data, using existing geoenabled county data.

In [41]:
item_counties = arcgis.gis.Item(arcgis.GIS(),'d8d3db28fe72445aa0449cfbcd6d0da3')
item_counties

<Item title:"US Counties and Equivalent Governmental Units" type:Feature Layer Collection owner:USFSMapsandApps>

In [43]:
lyr_counties = item_counties.layers[0]
counties_df = lyr_counties.query(out_fields = 'ST_CNTY_CODE, SHAPE',
                                  as_df = True)

counties_df.rename(columns = {'ST_CNTY_CODE':'GEOID'}, inplace=True)
counties_df.head()

,OBJECTID,SHAPE,GEOID
0,1,"{'rings': [[[-12138708.668, 5200561.206100002]...",56035
1,2,"{'rings': [[[-11080128.8702, 5388336.3829], [-...",46123
2,3,"{'rings': [[[-10495240.278, 5140480.516099997]...",19077
3,4,"{'rings': [[[-10893020.5821, 5495705.992899999...",46111
4,5,"{'rings': [[[-10872753.0909, 4209628.306500003...",40051


In [45]:
item_voterstats = arcgis.gis.Item(arcgis.GIS(),'2e8aaf91178c4c91b974d0bc4234dbfa')
df_voterstats = item_voterstats.layers[1].query(out_fields = 'GEOID, Total_cvap_est, GEONAME',
                                               return_geometry = False).sdf

df_voterstats

,OBJECTID,GEOID,Total_cvap_est,GEONAME
0,1,01001,40690,"Autauga County, Alabama"
1,2,01003,151770,"Baldwin County, Alabama"
2,3,01005,20375,"Barbour County, Alabama"
3,4,01007,17590,"Bibb County, Alabama"
4,5,01009,42430,"Blount County, Alabama"
...,...,...,...,...
3215,3216,72145,43335,"Vega Baja Municipio, Puerto Rico"
3216,3217,72147,7075,"Vieques Municipio, Puerto Rico"
3217,3218,72149,18270,"Villalba Municipio, Puerto Rico"
3218,3219,72151,27785,"Yabucoa Municipio, Puerto Rico"


In [48]:
counties_df = pandas.merge(counties_df, df_voterstats, on='GEOID')

The county geometry dataset includes various attributes. You will simplify the dataframe to only include the attributes that you need. The Total_cvap_est attribute represents the total population in each county that are of voting age for the year 2016.

In [37]:
# Modify the dataframe to only include the attributes that are needed
# counties_df = counties_df[['OBJECTID', 'GEOID', 'GEONAME',
#                            'Total_cvap_est',
#                            'SHAPE', 'Shape__Area', 'Shape__Length']]
# counties_df.head()

,OBJECTID,GEOID,GEONAME,Total_cvap_est,SHAPE,Shape__Area,Shape__Length
0,1,01001,"Autauga County, Alabama",40690,"{'rings': [[[-9619465, 3856529.0001000017], [-...",2.208654e+09,2.498864e+05
1,2,01003,"Baldwin County, Alabama",151770,"{'rings': [[[-9746859, 3539643.0001000017], [-...",5.671048e+09,1.655940e+06
2,3,01005,"Barbour County, Alabama",20375,"{'rings': [[[-9468394, 3771591.0001000017], [-...",3.257902e+09,3.208964e+05
3,4,01007,"Bibb County, Alabama",17590,"{'rings': [[[-9692114, 3928124.0001000017], [-...",2.311999e+09,2.279184e+05
4,5,01009,"Blount County, Alabama",42430,"{'rings': [[[-9623907, 4063676.0001000017], [-...",2.456909e+09,2.926429e+05


***

## Join the data

You have a dataframe with election data ('df_out') and a spatially-enabled dataframe of the county geometry data ('counties_df'). You will merge these datasets into one. 

In [49]:
# Join the election dataframe with the county geometry dataframe
geo_df = pandas.merge(df_out, counties_df, left_on='FIPS', right_on="GEOID", how='left')

# Visualize the merged data
geo_df.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,OBJECTID_x,SHAPE,GEOID,OBJECTID_y,Total_cvap_est,GEONAME
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,...,0.203059,0.693028,-0.203059,-0.693028,1941.0,"{'rings': [[[-9619464.5849, 3856529.2568999976...",01001,1.0,40690.0,"Autauga County, Alabama"
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,...,0.153169,0.724770,-0.153169,-0.724770,2690.0,"{'rings': [[[-9772603.0594, 3671612.347900003]...",01003,2.0,151770.0,"Baldwin County, Alabama"
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,...,0.451524,0.507212,-0.451524,-0.507212,1634.0,"{'rings': [[[-9468394.3046, 3771591.217299998]...",01005,3.0,20375.0,"Barbour County, Alabama"
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,...,0.189024,0.740560,-0.189024,-0.740560,1923.0,"{'rings': [[[-9692114.1573, 3928123.611299999]...",01007,4.0,17590.0,"Bibb County, Alabama"
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,...,0.061865,0.870955,-0.061865,-0.870955,522.0,"{'rings': [[[-9623906.6881, 4063676.209899999]...",01009,5.0,42430.0,"Blount County, Alabama"


The resulting dataframe includes the attributes from your election data and the specified attributes from the county geometry data. The SHAPE field represents the county geometry and is used to locate each record, or feature, on the map.

***

## Query and calculate attributes

Because you have the voting age population for 2016, you can now calculate the average voter participation (voter turnout) for 2016. The dataframe includes records from 2010-2016 but only has voting age population for 2016. You will need to create a subset dataframe for 2016 before calculating the voter turnout.

In [50]:
# Create a copy of the data, and perform a query
data_2016_df = geo_df.copy()
data_2016_df.query("year == '2016'", inplace=True)
data_2016_df.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,OBJECTID_x,SHAPE,GEOID,OBJECTID_y,Total_cvap_est,GEONAME
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,...,0.203059,0.693028,-0.203059,-0.693028,1941.0,"{'rings': [[[-9619464.5849, 3856529.2568999976...",01001,1.0,40690.0,"Autauga County, Alabama"
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,...,0.153169,0.724770,-0.153169,-0.724770,2690.0,"{'rings': [[[-9772603.0594, 3671612.347900003]...",01003,2.0,151770.0,"Baldwin County, Alabama"
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,...,0.451524,0.507212,-0.451524,-0.507212,1634.0,"{'rings': [[[-9468394.3046, 3771591.217299998]...",01005,3.0,20375.0,"Barbour County, Alabama"
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,...,0.189024,0.740560,-0.189024,-0.740560,1923.0,"{'rings': [[[-9692114.1573, 3928123.611299999]...",01007,4.0,17590.0,"Bibb County, Alabama"
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,...,0.061865,0.870955,-0.061865,-0.870955,522.0,"{'rings': [[[-9623906.6881, 4063676.209899999]...",01009,5.0,42430.0,"Blount County, Alabama"


In [51]:
data_2016_df.dtypes

FIPS                        object
year                         int64
county                      object
state                       object
state_po                    object
office                      object
candidate_dem               object
votes_dem                  float64
candidate_gop               object
votes_gop                  float64
votes_total                  int64
votes_other                float64
voter_share_major_party    float64
voter_share_dem            float64
voter_share_gop            float64
voter_share_other          float64
rawdiff_dem_vs_gop         float64
rawdiff_gop_vs_dem         float64
rawdiff_dem_vs_other       float64
rawdiff_gop_vs_other       float64
rawdiff_other_vs_dem       float64
rawdiff_other_vs_gop       float64
pctdiff_dem_vs_gop         float64
pctdiff_gop_vs_dem         float64
pctdiff_dem_vs_other       float64
pctdiff_gop_vs_other       float64
pctdiff_other_vs_dem       float64
pctdiff_other_vs_gop       float64
OBJECTID_x          

You will calculate a new field named voter turnout using field operators in Pandas. The operations will apply to all values across the columns. 

In [52]:
# Calculate voter turnout attributes
data_2016_df['voter_turnout'] = data_2016_df['votes_total'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_majparty'] = (data_2016_df['votes_dem']+data_2016_df['votes_gop']) / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_dem'] = data_2016_df['votes_dem'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_gop'] = data_2016_df['votes_gop'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_other'] = data_2016_df['votes_other'] / data_2016_df['Total_cvap_est']
data_2016_df.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,SHAPE,GEOID,OBJECTID_y,Total_cvap_est,GEONAME,voter_turnout,voter_turnout_majparty,voter_turnout_dem,voter_turnout_gop,voter_turnout_other
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,...,"{'rings': [[[-9619464.5849, 3856529.2568999976...",01001,1.0,40690.0,"Autauga County, Alabama",0.613738,0.592480,0.145884,0.446596,0.021258
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,...,"{'rings': [[[-9772603.0594, 3671612.347900003]...",01003,2.0,151770.0,"Baldwin County, Alabama",0.627364,0.601838,0.121618,0.480220,0.025525
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,...,"{'rings': [[[-9468394.3046, 3771591.217299998]...",01005,3.0,20375.0,"Barbour County, Alabama",0.513816,0.506748,0.239067,0.267681,0.007067
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,...,"{'rings': [[[-9692114.1573, 3928123.611299999]...",01007,4.0,17590.0,"Bibb County, Alabama",0.501364,0.489596,0.106538,0.383059,0.011768
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,...,"{'rings': [[[-9623906.6881, 4063676.209899999]...",01009,5.0,42430.0,"Blount County, Alabama",0.603064,0.589559,0.050813,0.538746,0.013505


***

## Validate the data

Before continuing with other data preparation, you should confirm that the output data has been successfully created. 

First, you will validate the values for voter turnout. You will remove null values, and because these values represent a fraction (total votes divided by voting age population), you will confirm that the values range between 0 and 1.

In [53]:
# Check for null values
data_2016_df.loc[data_2016_df['voter_turnout'].isnull()]

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,SHAPE,GEOID,OBJECTID_y,Total_cvap_est,GEONAME,voter_turnout,voter_turnout_majparty,voter_turnout_dem,voter_turnout_gop,voter_turnout_other
67,02701,2016,District 1,Alaska,AK,President,Hillary Clinton,2573.0,Donald Trump,3180.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,02702,2016,District 2,Alaska,AK,President,Hillary Clinton,1585.0,Donald Trump,3188.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,02703,2016,District 3,Alaska,AK,President,Hillary Clinton,1241.0,Donald Trump,5403.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,02704,2016,District 4,Alaska,AK,President,Hillary Clinton,4162.0,Donald Trump,4070.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,02705,2016,District 5,Alaska,AK,President,Hillary Clinton,3187.0,Donald Trump,3683.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,02706,2016,District 6,Alaska,AK,President,Hillary Clinton,2536.0,Donald Trump,4929.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,02707,2016,District 7,Alaska,AK,President,Hillary Clinton,1510.0,Donald Trump,5935.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,02708,2016,District 8,Alaska,AK,President,Hillary Clinton,1218.0,Donald Trump,6126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,02709,2016,District 9,Alaska,AK,President,Hillary Clinton,1843.0,Donald Trump,6100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,02710,2016,District 10,Alaska,AK,President,Hillary Clinton,1808.0,Donald Trump,6255.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# Remove records with no voter turnout value
data_2016_df = data_2016_df.loc[data_2016_df['voter_turnout'].notnull()]

In [55]:
# Run a describe to get the distribution of voter turnout values
data_2016_df['voter_turnout'].describe()

count    3111.000000
mean        0.594528
std         0.093493
min         0.158585
25%         0.530778
50%         0.595395
75%         0.656265
max         1.121277
Name: voter_turnout, dtype: float64

The describe function indicates that there are voter turnout values over one, indicating a voter turnout above 100%. You will further investigate by querying for these records.

In [56]:
# Perform query for voter turnout above 100%
data_2016_df.loc[data_2016_df['voter_turnout'] > 1]

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,SHAPE,GEOID,OBJECTID_y,Total_cvap_est,GEONAME,voter_turnout,voter_turnout_majparty,voter_turnout_dem,voter_turnout_gop,voter_turnout_other
311,08111,2016,San Juan,Colorado,CO,President,Hillary Clinton,265.0,Donald Trump,215.0,...,"{'rings': [[[-11964862.5883, 4528624.665600002...",08111,301.0,495.0,"San Juan County, Colorado",1.022222,0.969697,0.535354,0.434343,0.052525
1816,35021,2016,Harding,New Mexico,NM,President,Hillary Clinton,156.0,Donald Trump,311.0,...,"{'rings': [[[-11578221.842, 4330662.7557], [-1...",35021,1807.0,470.0,"Harding County, New Mexico",1.121277,0.993617,0.331915,0.661702,0.127660
2684,48301,2016,Loving,Texas,TX,President,Hillary Clinton,4.0,Donald Trump,58.0,...,"{'rings': [[[-11546376.1888, 3763338.364900000...",48301,2674.0,60.0,"Loving County, Texas",1.083333,1.033333,0.066667,0.966667,0.050000
2689,48311,2016,McMullen,Texas,TX,President,Hillary Clinton,40.0,Donald Trump,454.0,...,"{'rings': [[[-10984562.4854, 3330969.734399996...",48311,2679.0,460.0,"McMullen County, Texas",1.084783,1.073913,0.086957,0.986957,0.010870


There are four counties with very low population that resulted in voter turnout values above 100%. You could remove these records from the data or do additional research to identify the source of this issue. 

***

## Update validated data

After reviewing the Census Bureau voting age population data for 2016, you determined that these counties have a low voting age population with a fairly high margin of error. This may be the reason why these counties have a voter turnout rate higher than 100%. You will recalculate the voter turnout field for these counties using the upper range of their margin of error: 
- San Juan County, Colorado: 574
- Harding County, New Mexico: 562
- Loving County, Texas: 86
- McMullen County, Texas: 566

**Note: This information was extracted from this [table](https://data.census.gov/cedsci/table?q=voting%20age%20population%202016&g=0500000US08111,35021,48301,48311&hidePreview=true&table=DP05&tid=ACSDP5Y2016.DP05&t=Age%20and%20Sex&y=2016&lastDisplayedRow=6&vintage=2016&mode=&moe=true).**

In [57]:
# Correct each county
data_2016_df.loc[data_2016_df['FIPS'] == "08111", "Total_cvap_est"] = 574
data_2016_df.loc[data_2016_df['FIPS'] == "35021", "Total_cvap_est"] = 562
data_2016_df.loc[data_2016_df['FIPS'] == "48301", "Total_cvap_est"] = 86
data_2016_df.loc[data_2016_df['FIPS'] == "48311", "Total_cvap_est"] = 566

In [58]:
# Recalculate voter turnout fields
data_2016_df['voter_turnout'] = data_2016_df['votes_total'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_majparty'] = (data_2016_df['votes_dem']+data_2016_df['votes_gop']) / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_dem'] = data_2016_df['votes_dem'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_gop'] = data_2016_df['votes_gop'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_other'] = data_2016_df['votes_other'] / data_2016_df['Total_cvap_est']

To confirm that this correction addressed the issue, you will again query for counties with a voter turnout value above 100%.

In [59]:
data_2016_df.loc[data_2016_df['voter_turnout'] > 1]

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,SHAPE,GEOID,OBJECTID_y,Total_cvap_est,GEONAME,voter_turnout,voter_turnout_majparty,voter_turnout_dem,voter_turnout_gop,voter_turnout_other


No records are returned, indicating that there are no counties with a turnout value above 100%. Well done! You have cleaned the data. Next, you will convert the dataframe to a permanent dataset called a feature class. Feature classes are stored in an ArcGIS Pro file geodatabase.

***

## Convert dataframes to feature classes

You will use the ArcGIS API for Python, imported at the beginning of this script, to export the spatially-enabled dataframe to a feature class.

**Note: Executing the following cell may take a few minutes**

In [ ]:
fgdb = input('File Geodatabase: ')
# Create a feature class for the 2016 presidential election 
out_2016_fc_name = "county_elections_pres_2016"
out_2016_fc = data_2016_df.spatial.to_featureclass(os.path.join(fgdb, out_2016_fc_name))
out_2016_fc

1. At the top of the page, click the Data Engineering map tab.

2. Drag the Data Engineering map tab to display as its own window. 

3. Review the feature class that was added to the Data Engineering map.

![DataFrameToFeatureClass](img/DataFrameToFeatureClass.PNG "Map of counties, with missing county")

**Note: The color of the data will vary every time it is added to the map.** 


***

## Correct for missing data

The feature class is missing a county in South Dakota. You will correct this issue by further exploring the data.

1. In Catalog pane, expand Databases, and then Data Engineering and Visualization.gdb.
2. Right-click Counties_2016_VotingAgePopulation and choose Add To Current Map.
3. In the Contents pane, drag Counties_2016_VotingAgePopulation under county_elections_pres_2016.
4. Open the Data Engineering tab.
5. On the map, click the missing county.

![missing county](img/missing_county_view.PNG "Pop-up window for Oglala Lakota County")

The county geometry dataset identifies the missing county as Oglala Lakota County. By searching online for this county, you determine that Oglala Lakota County changed its county name and FIPS in 2015. It was originally Shannon County with a FIPS of 46113 and is now Oglala Lakota County with a FIPS of 46102. You will search the election data for the current FIPS to try to find the missing data.

In [60]:
# Perform query for county FIPS 46102
df_out.loc[df_out['FIPS'] == '46102']

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop


There are no records returned, which indicates that the election data does not have the correct FIPS for this county. You will check for the old FIPS value, when it was named Shannon County.

In [61]:
df_out.loc[df_out['FIPS'] == '46113']

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop
2428,46113,2016,Oglala Lakota,South Dakota,SD,President,Hillary Clinton,2510.0,Donald Trump,241.0,...,2356.0,87.0,-2356.0,-87.0,0.781067,-0.781067,0.811015,0.029948,-0.811015,-0.029948


There is the issue! The data has the correct name (Oglala Lakota) but the wrong FIPS (46113). You will correct this data issue.

In [62]:
df_out.loc[df_out['FIPS'] == '46113', 'FIPS'] = "46102"
df_out.loc[df_out['FIPS'] == '46102']

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,...,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop
2428,46102,2016,Oglala Lakota,South Dakota,SD,President,Hillary Clinton,2510.0,Donald Trump,241.0,...,2356.0,87.0,-2356.0,-87.0,0.781067,-0.781067,0.811015,0.029948,-0.811015,-0.029948


With the corrected FIPS value for Oglala Lakota County, you can now rejoin the geometry, recalculate the voting turnout field, and recreate the feature class. 

**Note: Executing the following cell may take a few minutes.**

In [64]:
# Join the county geometry data to the updated election data table
geo_df = pandas.merge(df_out, counties_df, left_on='FIPS', right_on="GEOID", how='left')

# Create a copy of the data that only includes records from 2016
data_2016_df = geo_df.copy()
data_2016_df.query("year == '2016'", inplace=True)
data_2016_df.head()

# Correct counties with low population
data_2016_df.loc[data_2016_df['FIPS'] == "08111", "Total_cvap_est"] = 574
data_2016_df.loc[data_2016_df['FIPS'] == "35021", "Total_cvap_est"] = 562
data_2016_df.loc[data_2016_df['FIPS'] == "48301", "Total_cvap_est"] = 86
data_2016_df.loc[data_2016_df['FIPS'] == "48311", "Total_cvap_est"] = 566

# Calculate voter turnout
data_2016_df['voter_turnout'] = data_2016_df['votes_total'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_majparty'] = (data_2016_df['votes_dem']+data_2016_df['votes_gop']) / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_dem'] = data_2016_df['votes_dem'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_gop'] = data_2016_df['votes_gop'] / data_2016_df['Total_cvap_est']
data_2016_df['voter_turnout_other'] = data_2016_df['votes_other'] / data_2016_df['Total_cvap_est']

# Remove records with no voter turnout value
data_2016_df = data_2016_df.loc[data_2016_df['voter_turnout'].notnull()]

You will export the dataframe to a feature class that you can visualize and analyze in ArcGIS Pro. 

**Note: Executing the following cell may take a few minutes.**

In [ ]:
# Create a feature class for the 2016 election and voter turnout data
fgdb = input('File Geodatabase: ')
out_2016_fc_name = "county_elections_pres_2016_final"
out_2016_fc = data_2016_df.spatial.to_featureclass(os.path.join(fgdb, out_2016_fc_name))

You have prepared this data for a predictive analysis that will model voter turnout using demographic variables, such as per capita income. In the next step, you will use ArcGIS Pro to geoenrich your feature class with these demographic variables. 

Open the Perform data engineering tasks exercise PDF and refer to the Open the Enrich tool step for the remaining instructions.

***